In [ ]:
!pip install qutip
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 19.7 MB/s eta 0:00:00


# Generation of spin-photon entangled pair via BK protocol


Based on the spin-photon entanglement result, we would like to develope the protocol to entangle the two spin systems with high fidelity. To start with, let's prepare two copies of spin-photon systems. In BK protocol (Phys. Rev. A 71, 060310 (2005)), there are 5 steps to achieve high fiedlity or highly entangled state and consists of 5 steps.

**Step 1:** Prepare the initial state as the equally suerposition state as: $|\psi_0\rangle=\frac{1}{\sqrt{2}}\left(|g_{\downarrow}0\rangle_A+|g_{\uparrow}0\rangle_A\right)\otimes\frac{1}{\sqrt{2}}\left(|g_{\downarrow}0\rangle_B+|g_{\uparrow}0\rangle_B\right)$.

**Step 2:** Send the optical field, whose Rabi frequency is $\Omega_s$ and is resonant to the frequency between the transition $|g_\downarrow\rangle_{s}\rightarrow|e\rangle_{s}$, to the NV system, where $s=A,B$. The interaction Hamiltonian can be written as: $$\hat{H}/\hbar=\sum_{s=A,B}\Omega_s\hat{a}_s\hat{\sigma}_{+,s}+\text{h.c.}$$
, where $\hat{\sigma}_{+,s}=|e\rangle_{ss}\langle g_{\downarrow}|$, with the excitation time-dependent Hamiltonian that uses the semi-classical approximation: $$\hat{H}_{\text{exc}}\left(t\right)/\hbar=\sum_{s=A,B}P_s\left(t\right)\hat{\sigma}_{+,s}+\text{h.c.}$$
, where $P_s\left(t\right)$ is the Rabi frequency of the excitation pulse. For simplicity, let's assume identical Rabi frequencies, excitation pulses of system A and B as $\Omega$ and $P\left(t\right)$.

**Step 3:** Counts the clicks that occur at detector $D_1$ or $D_2$. As a reminder, $D_1$ (or $D_2$) stands for the collapsing operator of mode $\hat{a}_A+\hat{a}_B$ (or $\hat{a}_A-\hat{a}_B$).

**Step 4:** Applying both qubits (A and B) with X-gate operation individually, $\hat{X}=\left(|g_\uparrow\rangle\langle g_\downarrow|+|g_\downarrow\rangle\langle g_\uparrow|+|e\rangle\langle e|\right)_A\otimes\left(|g_\uparrow\rangle\langle g_\downarrow|+|g_\downarrow\rangle\langle g_\uparrow|+|e\rangle\langle e|\right)_B$, to flip the spins.

**Step 5:** Repeat Step 1.

In the following, we provide the codes to simulate the overall process.

In [ ]:
Na = 3 # number of atomic levels
ground_down = basis(Na, 0)
ground_up = basis(Na, 1)
excited = basis(Na, 2)

# Here, let's choose the basis of state as |photon_A, spin_A, photon_B, spin_B>

N = 2 # Set where to truncate Fock state of cavity
sigma_A_gd_e = tensor(qeye(N), ground_down * excited.dag(), qeye(N), qeye(Na)) # |g_down><e| of system A
sigma_B_gd_e = tensor(qeye(N), qeye(Na), qeye(N), ground_down * excited.dag()) # |g_down><e| of system B
sigma_A_gd_gu = tensor(qeye(N), ground_down * ground_up.dag(), qeye(N), qeye(Na)) # |g_down><g_up| of system A
sigma_B_gd_gu = tensor(qeye(N), qeye(Na), qeye(N), ground_down * ground_up.dag()) # |g_down><g_up| of system B

a_A = tensor(destroy(N), qeye(Na), qeye(N), qeye(Na)) # annihiliation occurs of system A
a_B = tensor(qeye(N), qeye(Na), destroy(N), qeye(Na)) # annihiliation occurs of system B

### Let's define collapse operators
c_ops = [] # Build collapse operators

kappa =0.1 # Cavity decay rate
#kappa_loss=.09 # system loss rate
c_ops.append(np.sqrt(kappa) * a_A) # Cavity decay of system A. C0
c_ops.append(np.sqrt(kappa) * a_B) # Cavity decay of system B. C1
# c_ops.append(np.sqrt(kappa_loss) * a_A) # Cavity decay of system A. C0
# c_ops.append(np.sqrt(kappa_loss) * a_B) # Cavity decay of system B. C1




gamma =1 # Atomic decay rate
c_ops.append(np.sqrt(gamma) * sigma_A_gd_e) # spontaneous decay (decaying to modes other than cavity) from |e> to |g_down> of system A.   C2
c_ops.append(np.sqrt(gamma) * sigma_B_gd_e) # spontaneous decay (decaying to modes other than cavity) from |e> to |g_down> of system B.   C3

K_c = 20 # rate for collapsing to detector
c_ops.append(np.sqrt(K_c) * (a_A+a_B)/np.sqrt(2)) # Collapsing to detector 1        C4
c_ops.append(np.sqrt(K_c) * (a_A-a_B)/np.sqrt(2)) # Collapsing to detector 2        C5

X=ground_up*ground_down.dag()+ground_down*ground_up.dag()+excited*excited.dag()
X=tensor(qeye(N),X,qeye(N),X)

time_scale=1000
t = np.linspace(0.0, 5.0, time_scale) # Define time vector

# Step 1
theta=np.pi/4
photon = basis(N, 0) # Initial photonic state
spin = ground_down*np.cos(theta)+ground_up*np.sin(theta) # Initial spin state
psi0 = tensor(photon, spin, photon, spin) # Initial global state

g0 = 2 # coupling strength (Rabi frequency of vacuum field)
# Here describes the interaction Hamiltonian
H0_A = -g0 * (sigma_A_gd_e.dag() * a_A+a_A.dag()* sigma_A_gd_e) # time-independent Hamiltonian of system A
H0_B = -g0 * (sigma_B_gd_e.dag() * a_B+a_B.dag()* sigma_B_gd_e) # time-independent Hamiltonian of system B
H0 = H0_A + H0_B # time-independent Hermitian of global system

# Here describes the excitation Hamiltonian
H1_A = (sigma_A_gd_e.dag() + sigma_A_gd_e) # time-dependent Hamiltonian of system A after semi-classical approximation
H1_B = (sigma_B_gd_e.dag() + sigma_B_gd_e) # time-dependent Hamiltonian of system A after semi-classical approximation


numb=50 # numbers of trajectories
# ntraj=[numb]

# Excitation pulse parameters
center=0.5
life_time=0.04
peak =np.sqrt(np.pi)/2/life_time
excite_pulse = peak * np.exp(-((t-center) / life_time) ** 2)

#Step 2
H = [H0,[H1_A, excite_pulse],[H1_B, excite_pulse]]
output = mcsolve(H, psi0, t, c_ops, [], ntraj = numb, progress_bar=False)

/usr/local/lib/python3.10/dist-packages/qutip/solver/solver_base.py:459: FutureWarning: "progress_bar" is now included in options:
 Use `options={"progress_bar": False / True / "tqdm" / "enhanced"}`
  warnings.warn(


We count the clicks that occur on each detector, and run the monte carlo solver a second time, recording the average resulting states:

In [ ]:
# Here describes the excitation Hamiltonian of the microwave pi pulse after first trial
center=2
sig=0.4
excite_pulse2 = np.pi/2*(np.exp(-(t-center)**2/sig**2/2)/sig/np.sqrt(2*np.pi))

H2_A = (sigma_A_gd_gu.dag() + sigma_A_gd_gu) # time-dependent Hamiltonian of system A after semi-classical approximation
H2_B = (sigma_B_gd_gu.dag() + sigma_B_gd_gu) # time-dependent Hamiltonian of system A after semi-classical approximation
H_second=[[H2_A,excite_pulse2], [H2_B,excite_pulse2]]

#Helper function that applies an X rotation and runs mcsolve a second time, accumulating the results in the rhos list
def run_second_mcsolve(final_state, first_run_detector, rhos, counts):
    #Step 4
    # Let's apply the pi pulse to flip the two spins, which is equivalent to apply a deterministic X gate
    result0 = mesolve(H_second, final_state, t)
    psi0 = result0.states[-1]
    #Step 5
    result = mcsolve(H, psi0, t, c_ops, [], ntraj = numb, progress_bar=False)

    for i in range(numb):
        #For all trajectories, check if either detector fired, and if so add the final state to the appropriate index in rhos
        if 4 in result.col_which[i]:
            rho_num = 0 if first_run_detector == 4 else 2
            rhos[rho_num] += result.states[i][-1].ptrace([1, 3])
            counts[rho_num] += 1
        if 5 in result.col_which[i]:
            rho_num = 1 if first_run_detector == 4 else 3
            rhos[rho_num] += result.states[i][-1].ptrace([1, 3])
            counts[rho_num] += 1

#We store the cumulative final states for each combination of detector events in the order [D1->D1, D1->D2, D2->D1, D2->D2]
#We will then divide each of these cumulative states by counts to find the average final state for each combination
rhos = [0*output.states[0][0].ptrace([1,3]) for _ in range(4)]
counts = [0, 0, 0, 0]

for i in range(numb):
    #Step 3
    if 4 in output.col_which[i]:
        run_second_mcsolve(output.states[i][-1], 4, rhos, counts)
    if 5 in output.col_which[i]:
        run_second_mcsolve(output.states[i][-1], 5, rhos, counts)
rhos = [rhos[i] / counts[i] for i in range(4)]
succ_prob = [counts[i]/numb**2 for i in range(4)]

In [ ]:
# We expect the spin state that D1 fires in both rounds will result in a bell state |01>+|10> and let's calculate the entanglement entropy as well as their fiedlities
Bell1=tensor(ground_down,ground_up)/np.sqrt(2)+tensor(ground_up,ground_down)/np.sqrt(2)
print("BK scheme:  entanglement entropy (D1->D1): ",(entropy_vn(rhos[0].ptrace(0),2)+entropy_vn(rhos[0].ptrace(1),2))/2, " with fidelity:",fidelity(rhos[0],Bell1)," (|01>+10>) and success probability ",succ_prob[0])
# We expect the spin state that D1 fires in both rounds will result in a bell state |01>+|10> and let's calculate the entanglement entropy as well as their fiedlities
print("BK scheme:  entanglement entropy (D2->D2): ",(entropy_vn(rhos[3].ptrace(0),2)+entropy_vn(rhos[3].ptrace(1),2))/2, " with fidelity:",fidelity(rhos[3],Bell1)," (|01>+10>) and success probability ",succ_prob[1])
# We expect the spin state of the event that D1 fires in the first and D2 fires in the second round results in a bell state |01>-|10> and let's calculate the entanglement entropy as well as their fiedlities
Bell2=tensor(ground_down,ground_up)/np.sqrt(2)-tensor(ground_up,ground_down)/np.sqrt(2)
print("BK scheme:  entanglement entropy (D1->D2): ",(entropy_vn(rhos[1].ptrace(0),2)+entropy_vn(rhos[1].ptrace(1),2))/2, " with fidelity:",fidelity(rhos[1],Bell2)," (|01>-10>) and success probability ",succ_prob[2])
# We expect the spin state of the event that D2 fires in the first and D1 fires in the second round results in a bell state |01>-|10> and let's calculate the entanglement entropy as well as their fiedlities
print("BK scheme:  entanglement entropy (D2->D1): ",(entropy_vn(rhos[2].ptrace(0),2)+entropy_vn(rhos[2].ptrace(1),2))/2, " with fidelity:",fidelity(rhos[2],Bell2)," (|01>-10>) and success probability ",succ_prob[3])

BK scheme:  entanglement entropy (D1->D1):  0.9998848220847385  with fidelity: 0.996747486421616  (|01>+10>) and success probability  0.0308
BK scheme:  entanglement entropy (D2->D2):  0.999667329736152  with fidelity: 0.9944287787072384  (|01>+10>) and success probability  0.0328
BK scheme:  entanglement entropy (D1->D2):  0.9999043497579743  with fidelity: 0.9969459238887011  (|01>-10>) and success probability  0.0188
BK scheme:  entanglement entropy (D2->D1):  0.9970619379727941  with fidelity: 0.9839141113702974  (|01>-10>) and success probability  0.018
